# Importing Packages

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

# Accessing the URL

In [10]:
url = "https://books.toscrape.com/"
response = requests.get(url)

# Extraction of Data

In [ ]:
# results = soup.find(id="ResultsContainer")

# python_jobs = results.find_all(
#     "h2", string=lambda text: "python" in text.lower()
# )

# python_job_cards = [
#     h2_element.parent.parent.parent for h2_element in python_jobs
# ]
book_title = []
book_rating = []
book_price = []
dict_rating = {"One":1, "Two":2,"Three":3,"Four":4,"Five":5}
for page in range(1,4):
    url = f"https://books.toscrape.com/catalogue/page-{page}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    list_books = soup.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
    for book in list_books:
        book_title.append(book.find("a",title=True)["title"])
        book_rating.append(dict_rating[book.find("p",class_="star-rating")["class"][1]])
        book_price.append(book.find("div",class_="product_price").find("p",class_= "price_color").text[1:])
df = pd.DataFrame({
    "title": book_title,
    "price": np.float32(book_price),
    "rating": np.int8(book_rating)
})


## Filtering

In [84]:
df_filtered = df[(df["rating"] >= 4) & (df["price"] <= 30)]
df_filtered

,title,price,rating
8,The Boys in the Boat: Nine Americans and Their...,22.600000,4
11,Shakespeare's Sonnets,20.660000,4
12,Set Me Free,17.459999,5
23,Chase Me (Paris Nights #2),25.270000,5
30,The Four Agreements: A Practical Guide to Pers...,17.660000,5
32,The Elephant Tree,23.820000,5
34,Sophie's World,15.940000,5
43,#HigherSelfie: Wake Up Your Life. Free Your So...,23.110001,5
47,Untitled Collection: Sabbath Poems 2014,14.270000,4
53,This One Summer,19.490000,4


## Saving to .csv

In [ ]:
df_filtered.to_csv("4rated_price_lower30")

# Price per Genre

In [133]:
url = "https://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
header_genre = soup.find("div",class_="container-fluid page")
list_genre = [(header_genre.find_all("a",href=True)[i].text.strip().lower().replace(" ","-"),i) for i in range(2,52)]
book_price = []
book_genre = []
for genre,genre_id in list_genre:
    print(genre_id, end="\r")
    page = 1
    url = f"https://books.toscrape.com/catalogue/category/books/{genre}_{genre_id}/index.html"
    response = requests.get(url)
    while response.status_code == 200:
    
        soup = BeautifulSoup(response.content, "html.parser")
        list_books = soup.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
        for book in list_books:
            book_price.append(book.find("div",class_="product_price").find("p",class_= "price_color").text[1:])
            book_genre.append(genre)
        page += 1 
        url = f"https://books.toscrape.com/catalogue/category/books/{genre}_{genre_id}/page-{page}.html"
        response = requests.get(url)
df_genre = pd.DataFrame({
    "genre": book_genre,
    "price": np.float32(book_price)
})

In [134]:
df_genre

,genre,price
0,travel,45.169998
1,travel,49.430000
2,travel,48.869999
3,travel,36.939999
4,travel,37.330002
...,...,...
995,politics,52.650002
996,politics,56.860001
997,cultural,36.580002
998,erotica,19.190001


## Top 10 Priciest Book Genres

In [136]:
df_price_per_genre = df_genre.groupby(by="genre").mean().sort_values(by="price",ascending=False)
df_price_per_genre[:10]

,price
genre,
suspense,58.330002
novels,54.810001
politics,53.613331
health,51.452499
new-adult,46.383331
christian,42.496670
sports-and-games,41.166000
self-help,40.620003
travel,39.794544
